In [ ]:
// Google Colab only: uncomment and run the following to install dotnet and the F# kernel
// !bash <(curl -Ls https://raw.githubusercontent.com/gbaydin/scripts/main/colab_dotnet6.sh)


In [ ]:
// Import Furnace package
#r "nuget: Furnace-lite,1.0.9"
#r "nuget: SixLabors.ImageSharp,1.0.1"

// Set dotnet interactive formatter to plaintext
Formatter.SetPreferredMimeTypesFor(typeof<obj>, "text/plain")
Formatter.Register(fun (x:obj) (writer: TextWriter) -> fprintfn writer "%120A" x )


[![Binder](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fsprojects/Furnace/blob/master/quickstart.ipynb)&emsp;
[![Script](img/badge-script.svg)](quickstart.fsx)&emsp;
[![Script](img/badge-notebook.svg)](quickstart.ipynb)

# Quickstart

Here we cover some key tasks involved in a typical machine learning pipeline and how these can be implemented with Furnace. Note that a significant part of Furnace's design has been influenced by [PyTorch](https://pytorch.org/) and you would feel mostly at home if you have familiarity with PyTorch.

## Datasets and Data Loaders

Furnace provides the [Dataset](https://fsprojects.github.io/Furnace/reference/furnace-data-dataset.html) type that represents a data source and the [DataLoader](https://fsprojects.github.io/Furnace/reference/furnace-data-dataloader.html) type that handles the loading of data from datasets and iterating over [minibatches](https://en.wikipedia.org/wiki/Stochastic_gradient_descent#Iterative_method) of data.

See the [Furnace.Data](/Furnace/reference/furnace-data.html) namespace for the full API reference.

### Datasets

Furnace has ready-to-use types that cover main datasets typically used in machine learning, such as [MNIST](https://fsprojects.github.io/Furnace/reference/furnace-data-mnist.html), [CIFAR10](https://fsprojects.github.io/Furnace/reference/furnace-data-cifar10.html), [CIFAR100](https://fsprojects.github.io/Furnace/reference/furnace-data-cifar100.html), and also more generic dataset types such as [TensorDataset](https://fsprojects.github.io/Furnace/reference/furnace-data-tensordataset.html) or [ImageDataset](https://fsprojects.github.io/Furnace/reference/furnace-data-imagedataset.html).

The following loads the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset and shows one image entry and the corresponding label.



In [2]:
open Furnace
open Furnace.Data

// First ten images in MNIST training set
let dataset = MNIST("../data", train=true, transform=id, n=10)

// Inspect a single image and label
let data, label = dataset[7]

// Save image to file
data.saveImage("test.png")


No value returned by any evaluator

In [4]:
// Inspect data as ASCII and show label
printfn "Data: %A\nLabel: %A" (data.toImageString()) label


Data: "                                                                                                                                                       ~-}@#####Z                -j*W########J'              O############i              [##Mxxxxo####i               ::^    'W##Z                       |&##f                      (o###'                    (q%###d.               "uaaa####8}:               _m########O                 _*####@####?                 "v<____f##?                        `##?                        |##?               ?.      1&##?             iQ#:    `)8##&!             p##txxxxb###o\              p#########MC.               +J#####wdt_                   }B#Z}^                                                                                                   "Label: tensor(3,dtype=Int32,device=cpu-1)

### Data Loaders

A data loader handles tasks such as constructing minibatches from an underlying dataset on-the-fly, shuffling the data, and moving the data tensors between devices. In the example below we show a single batch of six MNIST images and their corresponding classification labels.



In [5]:
let loader = DataLoader(dataset, shuffle=true, batchSize=6)
let batch, labels = loader.batch()

printfn "%A\nLabels: %A" (batch.toImageString()) labels


In practice a data loader is typically used to iterate over all minibatches in a given dataset in order to feed each minibatch through a machine learning model. One full iteration over the dataset would be called an "epoch". Typically you would perform multiple such epochs of iterations during the training of a model.



In [6]:
for epoch = 1 to 10 do
    for i, data, labels in loader.epoch() do
        printfn "Epoch %A, minibatch %A" epoch (i+1)
        // Process the minibatch
        // ...


## Models

Many machine learning models are differentiable functions whose parameters can be tuned via [gradient-based optimization](https://en.wikipedia.org/wiki/Gradient_descent), finding an optimum for an objective function that quantifies the fit of the model to a given set of data. These models are typically built as compositions non-linear functions and ready-to-use building blocks such as linear, recurrent, and convolutional layers.

Furnace provides the most commonly used model building blocks including convolutions, transposed convolutions, batch normalization, dropout, recurrent and other architectures.

See the [Furnace.Model](/Furnace/reference/furnace-model.html) namespace for the full API reference.

### Constructing models, PyTorch style

If you have experience with [PyTorch](https://pytorch.org/), you would find the following way of model definition familiar. Let's look at an example of a [generative adversarial network (GAN)](https://arxiv.org/abs/1406.2661) architecture.



In [7]:
open Furnace.Model
open Furnace.Compose

// PyTorch style

// Define a model class inheriting the base
type Generator(nz: int) =
    inherit Model()
    let fc1 = Linear(nz, 256)
    let fc2 = Linear(256, 512)
    let fc3 = Linear(512, 1024)
    let fc4 = Linear(1024, 28*28)
    do base.addModel(fc1, fc2, fc3, fc4)
    override self.forward(x) =
        x
        |> FurnaceImage.view([-1;nz])
        |> fc1.forward
        |> FurnaceImage.leakyRelu(0.2)
        |> fc2.forward
        |> FurnaceImage.leakyRelu(0.2)
        |> fc3.forward
        |> FurnaceImage.leakyRelu(0.2)
        |> fc4.forward
        |> FurnaceImage.tanh

// Define a model class inheriting the base
type Discriminator(nz:int) =
    inherit Model()
    let fc1 = Linear(28*28, 1024)
    let fc2 = Linear(1024, 512)
    let fc3 = Linear(512, 256)
    let fc4 = Linear(256, 1)
    do base.addModel(fc1, fc2, fc3, fc4)
    override self.forward(x) =
        x
        |> FurnaceImage.view([-1;28*28])
        |> fc1.forward
        |> FurnaceImage.leakyRelu(0.2)
        |> FurnaceImage.dropout(0.3)
        |> fc2.forward
        |> FurnaceImage.leakyRelu(0.2)
        |> FurnaceImage.dropout(0.3)
        |> fc3.forward
        |> FurnaceImage.leakyRelu(0.2)
        |> FurnaceImage.dropout(0.3)
        |> fc4.forward
        |> FurnaceImage.sigmoid

// Instantiate the defined classes
let nz = 128
let gen = Generator(nz)
let dis = Discriminator(nz)

print gen
print dis


### Constructing models, Furnace style

A key advantage of Furnace lies in the [functional programming](https://en.wikipedia.org/wiki/Functional_programming) paradigm enabled by the F# language, where functions are first-class citizens, many algorithms can be constructed by applying and composing functions, and differentiation operations can be expressed as composable [higher-order functions](https://en.wikipedia.org/wiki/Higher-order_function). This allows very succinct (and beautiful) machine learning code to be expressed as a powerful combination of [lambda calculus](https://en.wikipedia.org/wiki/Lambda_calculus) and [differential calculus](https://en.wikipedia.org/wiki/Differential_calculus).

For example, the following constructs the same GAN architecture (that we constructed in PyTorch style in the previous section) using Furnace's `-->` composition operator, which allows you to seamlessly compose `Model` instances and differentiable `Tensor->Tensor` functions.



In [8]:
// Furnace style

// Model as a composition of models and Tensor->Tensor functions
let generator =
    FurnaceImage.view([-1;nz])
    --> Linear(nz, 256)
    --> FurnaceImage.leakyRelu(0.2)
    --> Linear(256, 512)
    --> FurnaceImage.leakyRelu(0.2)
    --> Linear(512, 1024)
    --> FurnaceImage.leakyRelu(0.2)
    --> Linear(1024, 28*28)
    --> FurnaceImage.tanh

// Model as a composition of models and Tensor->Tensor functions
let discriminator =
    FurnaceImage.view([-1; 28*28])
    --> Linear(28*28, 1024)
    --> FurnaceImage.leakyRelu(0.2)
    --> FurnaceImage.dropout(0.3)
    --> Linear(1024, 512)
    --> FurnaceImage.leakyRelu(0.2)
    --> FurnaceImage.dropout(0.3)
    --> Linear(512, 256)
    --> FurnaceImage.leakyRelu(0.2)
    --> FurnaceImage.dropout(0.3)
    --> Linear(256, 1)
    --> FurnaceImage.sigmoid

print generator
print discriminator
